In [19]:
import numpy as np 
import os 
import pandas as pd 
import re
import string
import seaborn as sns
import sklearn
from surprise import Reader, Dataset, SVD, KNNBasic

from surprise.accuracy import rmse


from surprise.model_selection import cross_validate
from surprise.model_selection import train_test_split
import surprise
from surprise import KNNWithMeans
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import GridSearchCV

In [20]:
df_anime = pd.read_csv("../data/df_anime_export.csv")
df_profile = pd.read_csv("../data/df_profile_export.csv")
df_review = pd.read_csv("../data/df_review_export.csv")
df_an_pro_rev = pd.read_csv("../data/merged_rev_df_export.csv")
df_an_pro = pd.read_csv("../data/merged_df_export.csv")
df_main = pd.read_csv("../data/df_prof_rev_anime_clean.csv")

In [21]:
df_main.head(1)

,Unnamed: 0.1,uid,title,synopsis,genre,aired,episodes,members,popularity,ranked,score,synopsis_processed,profile,review_score,review_score_indepth,text_preprocessed,Unnamed: 0,gender,birthday,favorites_anime
0,0,28891,Haikyuu!! Second Season,Following their participation at the Inter-Hig...,"['Comedy', 'Sports', 'Drama', 'School', 'Shoun...","Oct 4, 2015 to Mar 27, 2016",25.0,489888,141,25.0,8.82,"following participation inter-high , karasuno ...",skrn,7,"{'Overall': '7', 'Story': '7', 'Animation': '9...","art : great , especially action critical momen...",2,NaN,NaN,"['918', '2904', '11741', '17074', '23273', '32..."


## Collab Filter a bit cleaner

In [22]:
df_touse = df_main[["profile", "uid", "review_score"]]

In [23]:
df_anime.head(1)

,Unnamed: 0,uid,title,synopsis,genre,aired,episodes,members,popularity,ranked,score,synopsis_processed
0,0,28891,Haikyuu!! Second Season,Following their participation at the Inter-Hig...,"['Comedy', 'Sports', 'Drama', 'School', 'Shoun...","Oct 4, 2015 to Mar 27, 2016",25.0,489888,141,25.0,8.82,"following participation inter-high , karasuno ..."


In [24]:
df_anime_touse = df_anime[["uid", "title"]]

In [25]:
df = Dataset.load_from_df(df_touse, Reader(rating_scale=(1,10)))
trainset = df.build_full_trainset()

In [29]:
model = SVD()
cross_validate(model, df, measures=["RMSE", "MAE"], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.9719  1.9774  1.9759  1.9823  1.9948  1.9804  0.0079  
MAE (testset)     1.5215  1.5241  1.5203  1.5335  1.5337  1.5266  0.0059  
Fit time          1.56    1.38    1.51    1.42    1.50    1.47    0.07    
Test time         0.17    0.37    0.16    0.16    0.36    0.24    0.10    


{'test_rmse': array([1.97187106, 1.97737511, 1.97587302, 1.98233606, 1.9947752 ]),
 'test_mae': array([1.52145495, 1.52406345, 1.5202852 , 1.53352173, 1.53372599]),
 'fit_time': (1.559349775314331,
  1.3801090717315674,
  1.5131442546844482,
  1.4203205108642578,
  1.5003361701965332),
 'test_time': (0.1710371971130371,
  0.3680858612060547,
  0.15903496742248535,
  0.1600334644317627,
  0.3600797653198242)}

In [30]:
#single prediction
model.predict('skrn', 30)

Prediction(uid='skrn', iid=30, r_ui=None, est=6.184150466905168, details={'was_impossible': False})

In [31]:
#overall highest predicted not seen shows based on whats seen
user_id = 'skrn'
all_anime = df_touse['uid'].unique()
watched = df_touse[df_touse['profile']==user_id].uid
not_watched = [anime for anime in all_anime if anime not in watched]

# predict
score = [model.predict(user_id, anime_id) for anime_id in not_watched]
anime_id = []
pred_score = []
for i in range(0, len(score)):
    anime_id.append(score[i].iid)
    pred_score.append(score[i].est)
df_pred = pd.DataFrame({'uid':anime_id, 'pred_score':pred_score})

df_pred_real = df_pred.sort_values('pred_score', ascending=False).head(10)

df_pred_real.merge(df_anime_touse, how='left', on='uid')

,uid,pred_score,title
0,918,8.856259,Gintama
1,5114,8.793556,Fullmetal Alchemist: Brotherhood
2,35247,8.651724,Owarimonogatari 2nd Season
3,7311,8.567288,Suzumiya Haruhi no Shoushitsu
4,28977,8.566027,Gintama°
5,3297,8.554141,Aria The Origination
6,9756,8.538318,Mahou Shoujo Madoka★Magica
7,4181,8.486295,Clannad: After Story
8,34599,8.460276,Made in Abyss
9,15335,8.414477,Gintama Movie 2: Kanketsu-hen - Yorozuya yo Ei...


In [ ]:
df_touse

,profile,uid,review_score
0,skrn,28891,7
1,skrn,32935,8
2,skrn,30276,7
3,skrn,4107,7
4,skrn,4081,4
...,...,...,...
130514,sensei69,33082,5
130515,ssjtk,33082,8
130516,Ground_zero,33082,9
130517,ClawViper,33082,7


In [ ]:
df_touse['uid'][0]

28891

## Testing Accuracy

In [ ]:
reader = Reader(rating_scale=(1, 10))
data = Dataset.load_from_df(df_touse, reader)

# Build the full training set
trainset = data.build_full_trainset()

# Train the SVD model
model = SVD()
model.fit(trainset)

# Split the dataset into training and testing sets
trainset, testset = train_test_split(data, test_size=0.2)

# Chunk the testset into groups of 1000
chunk_size = 1000
num_chunks = len(testset) // chunk_size
if len(testset) % chunk_size != 0:
    num_chunks += 1

# Calculate accuracy for each chunk
accuracies = []
tolerance = 1  # Tolerance for correctness

for i in range(num_chunks):
    start_idx = i * chunk_size
    end_idx = start_idx + chunk_size
    chunk = testset[start_idx:end_idx]
    predictions = model.test(chunk)

    # Calculate accuracy for the chunk
    num_correct = 0
    for prediction in predictions:
        uid = prediction.uid
        iid = prediction.iid
        actual_rating = prediction.r_ui
        predicted_rating = round(prediction.est)
        if abs(actual_rating - predicted_rating) <= tolerance:
            num_correct += 1

    # Calculate accuracy for the chunk
    accuracy = num_correct / len(chunk)
    accuracies.append(accuracy)

# Calculate the average accuracy across all chunks
average_accuracy = np.mean(accuracies)

print('Average Accuracy:', average_accuracy)

Average Accuracy: 0.8899373219373222
